### Water Pouring Problem 

In [3]:
# ---------------------------------------
# glass: capacity, current level
# a pair of glass:
# goal: try to reach
# pouring actions: empty, fill, transfer
# solution: sequences of actions
#
# successor: a collection of states that you can reach
# ---------------------------------------
def pour_problem(X, Y, goal, start = (0, 0)):
        """X and Y are thr capacity of glasses; (x, y) is current fill levels
        and represents a state. The goal is a level that can be in either glass.
        Start at start state and following successors until we reach the goal.
        Keep track of frontier and previously explored; fail when no frontier."""
        if goal is start:
            return [start]
        explored = set() # set of states we have visited
        frontier = [ [ start] ] # ordered list of paths we have blazed
        while frontier:
            path = frontier.pop(0)
            (x, y) = path[-1] # last state in the first path of the frontier
            for (state, action) in successors(x, y, X, Y).items():
                if state not in explored:
                    explored.append(state) 
                    path2 = path + [action, state]
                    if goal is state:
                        return path2
                    else:
                        frontier.append(path2)
        return Fail

def successors(x, y, X, Y):
    """Return a dict of {state: action} pairs describing what can be reached from the (x, y)
    state, and how."""
    assert x <= X and y <= Y ##(x, y) is glass level; X and Y are glass sizes
    return {((0, y + x) if y + x <= Y else(x - (Y - y), y + (Y - y))): 'X --> Y',
            ((x + y, 0) if y + x <= X else(x + (X - x), y - (X - x))): 'X <-- Y',
            (X, y): 'fill X', (x, Y): 'fill Y',
            (0, y): 'empty X', (x, 0): 'empty Y'}
    
Fail = []

In [4]:
import doctest  

### Bridge Problem 

In [8]:
#--------------------------------------
# people: 4 
# speed: 1min, 2min, 5min, 10min
# The bridge only can across two people one time
# collection of people: here, there
# tool: torch or the light
# state: (here, there, t)
#--------------------------------------

# -----------------
# User Instructions
# 
# Write a function, bsuccessors(state), that takes a state as input
# and returns a dictionary of {state:action} pairs.
#
# A state is a (here, there, t) tuple, where here and there are 
# frozensets of people (indicated by their times), and potentially
# the 'light,' t is a number indicating the elapsed time.
#
# An action is a tuple (person1, person2, arrow), where arrow is 
# '->' for here to there or '<-' for there to here. When only one 
# person crosses, person2 will be the same as person one, so the
# action (2, 2, '->') means that the person with a travel time of
# 2 crossed from here to there alone.

def bsuccessors(state):
    """Return a dict of {state:action} pairs. A state is a (here, there, t) tuple,
    where here and there are frozensets of people (indicated by their times) and/or
    the 'light', and t is a number indicating the elapsed time. Action is represented
    as a tuple (person1, person2, arrow), where arrow is '->' for here to there and 
    '<-' for there to here."""
    here, there, t = state
    if 'light' in here:
        return dict(((here - frozenset([a, b, 'light']),
                      there | frozenset([a, b, 'light']),
                      t + max(a,b)), (a, b, '->'))
                    for a in here if a is not 'light'
                    for b in here if b is not 'light')
    else:
        return dict(((here | frozenset([a, b, 'light']),
                      there - frozenset([a, b, 'light']),
                      t + max(a,b)), (a, b, '<-'))
                    for a in there if a is not 'light'
                    for b in there if b is not 'light')

def test():

    assert bsuccessors((frozenset([1, 'light']), frozenset([]), 3)) == {
                (frozenset([]), frozenset([1, 'light']), 4): (1, 1, '->')}

    assert bsuccessors((frozenset([]), frozenset([2, 'light']), 0)) =={
                (frozenset([2, 'light']), frozenset([]), 2): (2, 2, '<-')}
    
    return 'tests pass'

print (test())


tests pass
